In [ ]:
import sys
sys.path.append("..")
from figutils import *
import matplotlib.patheffects as pe

In [ ]:
scene_name = 'stream'
method = 'cv_ps'
output_dir = os.path.join(OUTPUT_DIR, scene_name)
iterations = [3, 5, 25, 75, 250, 750]

In [ ]:

imgs = []
weights = []
labels = []

method_dir = os.path.join(output_dir, method)

for i in iterations:
    labels.append(f"it={i}")
    imgs.append(mi.Bitmap(os.path.join(method_dir, "img", f"{i:04d}.exr")))
    weights.append(mi.Bitmap(os.path.join(output_dir, method, "weights", f"{i:04d}.exr")))

labels.append("Target")
imgs.append(mi.Bitmap(os.path.join(output_dir, "img_ref.exr")))
weights.append(None)
img_ref = mi.Bitmap(os.path.join(output_dir, "img_ref.exr"))
result = np.load(os.path.join(method_dir, "result.npz"))

variance = np.load(os.path.join(method_dir, "result.npz"))['var']

In [ ]:
sns.set_style('white')

In [ ]:
var = [variance[it] for it in iterations] 

In [ ]:
base_size = 4

w,h = imgs[0].size()

n_cols = len(imgs) + 1
n_rows = 2 if show_weights else 1

wspace = 0.03
hspace = 0.04 * w/h
aspect = gridspec_aspect(n_rows, n_cols, w, h, wspace=wspace, hspace=hspace)

fig = plt.figure(1, figsize=(PAGE_WIDTH, PAGE_WIDTH / aspect))

gs = fig.add_gridspec(n_rows, n_cols, wspace=wspace, hspace=hspace)

for i, (img, weight, label) in enumerate(zip(imgs, weights, labels)):
    ax = fig.add_subplot(gs[0, i])
    ax.imshow(mi.util.convert_to_bitmap(img), interpolation='none')
    disable_ticks(ax)
    if i != n_cols - 2:
        v = np.sqrt(var[i])
        e = int(np.log10(v))-1
        text = f"$\sigma={v:.3f}$"
        ax.text(0.99*w, 0.99*h, text, ha="right", va="bottom", color="white", fontsize=DEFAULT_FONTSIZE, path_effects=[pe.withStroke(linewidth=1, foreground="black")])

    if show_weights:
        if i == 0:
            ax.set_ylabel("Primal")
        ax = fig.add_subplot(gs[1, i])
        if weight is not None:
            weight = mi.TensorXf(weights[i])[:,:,0]
            im = ax.imshow(weight, cmap='Reds_r', vmin=0, vmax=1, interpolation='none')
        else:
            weight = np.ones((h, w))
            ax.imshow(weight, cmap='gray', vmin=0, vmax=1)
            ax.text(w/2, h/2, "N/A", ha="center", va="center", color="darkgrey")
            disable_border(ax)

        disable_ticks(ax)
        
        if i == 0:
            ax.set_ylabel("Weights")

    if i != n_cols-2:
        title = f"Step {iterations[i]}\n"
        ax.set_title(title, verticalalignment='top', y=-0.2)
    else:
        ax.set_title(label, verticalalignment='top', y=-0.2)

    if i == n_cols - 2:
        cbax = ax.inset_axes([1.04, 0, 0.05, 1], transform=ax.transAxes)
        cbar = fig.colorbar(im, cax=cbax, ticks=[0, 0.5, 1])
        cbar.outline.set_visible(False)
        cbar.ax.tick_params(size=0)

save_fig("stream")
